In [1]:
import pymongo
import pprint
from collections import OrderedDict
import pandas as pd

# Football Data Transfermarket

In [2]:
password = 'o&FaL^bYun$7'

# Connection Database

In [3]:
client = pymongo.MongoClient("mongodb+srv://Minetto:"+ password + "@cluster1.tvb8l.mongodb.net/test?retryWrites=true&w=majority")
db = client.football_stats2

In [5]:
pip install pymongo[srv]

Note: you may need to restart the kernel to use updated packages.


# Load Competition

In [7]:
header = ["competition_id","name","type","country_id","country_name","domestic_league_code","confederation","url"]
dtypes = {"competition_id": 'str',"name": 'str',"type": 'str',"country_name": 'str',"domestic_league_code": 'str',"confederation": 'str',"url": 'str'}
df = pd.read_csv(r"E:\magistrale\Data Managment\files\competitions.csv", names=header, dtype=dtypes)

In [8]:
df['_id'] = df['competition_id']

In [11]:
df.head(2)

,name,type,country_id,country_name,domestic_league_code,confederation,url,_id
0,name,type,country_id,country_name,domestic_league_code,confederation,url,competition_id
1,bundesliga,first_tier,40,Germany,L1,europa,https://www.transfermarkt.co.uk/bundesliga/sta...,L1


In [10]:
df.drop('competition_id', axis=1, inplace=True)

In [12]:
comp_dict = df.to_dict('records')

In [5]:
collection = db['competition']

In [13]:
collection.insert_many(comp_dict)

In [6]:
collection.delete_many({})

In [106]:
db.competition.count_documents({})

43

# Load Club

In [30]:
json =  {"$jsonSchema": {
			"required": [ "name"],
			"properties": {
				"name": { "bsonType": "string" },
				"pretty_name": { "bsonType": "string" },
				"domestic_competition_id": { "bsonType": "string" }
				
			} 
		}
    }

In [31]:
cmd = OrderedDict([('collMod', 'clubs'),
            ('validator', json),
            ('validationLevel', 'moderate')])

In [32]:
db.command(cmd)

{'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1653129693, 22),
  'signature': {'hash': b'\x0c\xaffGV\x02\x9f\xcc\xb5\xa8\xf1\x01\xc4[\x05\x83\x18qE[',
   'keyId': 7045684881749180491}},
 'operationTime': Timestamp(1653129693, 22)}

In [27]:
collection = db['clubs']

In [101]:
df = pd.read_csv(r"E:\magistrale\Data Managment\files\clubs.csv")

In [29]:
df['_id'] = df['club_id']
df.drop('club_id', axis=1, inplace=True)

In [30]:
df.head(2)

,name,pretty_name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,url,_id
0,ud-almeria,Ud Almeria,ES1,51.66,25,25.7,10,40.0,2,Estadio de los Juegos Mediterráneos,15274,£-4.82m,Francisco Rodríguez,https://www.transfermarkt.co.uk/ud-almeria/sta...,3302
1,niki-volou,Niki Volou,GR1,3.40,24,28.0,6,25.0,1,Panthessaliko Stadio,22700,+-0,Wiljan Vloet,https://www.transfermarkt.co.uk/niki-volou/sta...,5572


In [102]:
club_dict = df.to_dict('records')

In [ ]:
club_dict

In [32]:
collection.insert_many(club_dict)

In [16]:
collection.delete_many({})

# Load Games

In [95]:
games = pd.read_csv(r"E:\magistrale\Data Managment\hw3\games.csv")

In [96]:
games.head(1)

,name,game_id,competition_code,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,away_club_position,home_club_name,away_club_name
0,johan-cruijff-schaal,2457642,NLSC,2014,Final,2014-08-03,1269,610,1,0,0,0,Pec Zwolle,Ajax Amsterdam


In [98]:
#games['attendance'] = games['attendance'].fillna(0).astype("int64")
games['home_club_position'] = games['home_club_position'].fillna(0).astype("int64")
games['away_club_position'] = games['away_club_position'].fillna(0).astype("int64")
games['_id'] = games['game_id']
games.drop('game_id', axis=1, inplace=True)

In [99]:
collection = db['games']

In [101]:
games_dict = games.to_dict('records')

In [103]:
collection.insert_many(games_dict)

In [102]:
collection.delete_many({})

# Load Competition with game_id

In [4]:
comp = pd.read_csv(r"E:\magistrale\Data Managment\hw3\competitions.csv")

In [5]:
comp["_id"] = comp.competition_id

In [6]:
comp.drop('competition_id', axis=1, inplace=True)

In [7]:
comp.head()

,name,type,country_id,country_name,domestic_league_code,confederation,url,_id
0,bundesliga,first_tier,40,Germany,L1,europa,https://www.transfermarkt.co.uk/bundesliga/sta...,L1
1,dfb-pokal,domestic_cup,40,Germany,L1,europa,https://www.transfermarkt.co.uk/dfb-pokal/star...,DFB
2,dfl-supercup,domestic_super_cup,40,Germany,L1,europa,https://www.transfermarkt.co.uk/dfl-supercup/s...,DFL
3,eredivisie,first_tier,122,Netherlands,NL1,europa,https://www.transfermarkt.co.uk/eredivisie/sta...,NL1
4,toto-knvb-beker,domestic_cup,122,Netherlands,NL1,europa,https://www.transfermarkt.co.uk/toto-knvb-beke...,NLP


In [8]:
comp_dict = comp.to_dict('records')

In [89]:
games = pd.read_csv(r"E:\magistrale\Data Managment\hw3\games.csv")

In [21]:
comp_dict[0]

{'name': 'bundesliga',
 'type': 'first_tier',
 'country_id': 40,
 'country_name': 'Germany',
 'domestic_league_code': 'L1',
 'confederation': 'europa',
 'url': 'https://www.transfermarkt.co.uk/bundesliga/startseite/wettbewerb/L1',
 '_id': 'L1'}

In [11]:
idx_comp = dict()
for i, el in enumerate(comp_dict):
    idx_comp[el['_id']] = i
    comp_dict[i]['games'] = []

In [13]:
for i, row in games.iterrows():
    comp_dict[idx_comp[row['competition_code']]]['games'].append(row['game_id'])

In [14]:
collection = db['competition']

In [17]:
collection.delete_many({})

In [18]:
collection.insert_many(comp_dict)

# Load Player with Appearances

In [41]:
player = pd.read_csv(r"E:\magistrale\Data Managment\hw3\players.csv")


In [42]:
player['_id'] = player['player_id']
player.drop('player_id', inplace=True, axis=1)

In [43]:
play_dict = player.to_dict('records')
app = pd.read_csv(r"E:\magistrale\Data Managment\hw3\app1.csv")

In [44]:
idx_player = dict()
for i, el in enumerate(play_dict):
    idx_player[el['_id']] = i
    play_dict[i]['appearances'] = []
    play_dict[i]['all_club_id_player'] = []

In [65]:
app.head(1)

,season,name,player_id,game_id,appearance_id,competition_id,player_club_id,goals,assists,minutes_played,yellow_cards,red_cards
0,2014,serie-a,10,2478592,2478592_10,IT1,398,0,0,59,0,0


In [45]:
for i, row in app.iterrows():
    play_dict[idx_player[row['player_id']]]['appearances'].append(row.to_dict())
    if row['player_club_id'] not in play_dict[idx_player[row['player_id']]]['all_club_id_player']:
        play_dict[idx_player[row['player_id']]]['all_club_id_player'].append(row['player_club_id'])

In [47]:
import json
with open(r"E:\magistrale\Data Managment\hw3\prova", 'w') as fout:
    json.dump(play_dict, fout)

In [55]:
db = client.football_stats2

In [56]:
collection = db['players']

In [57]:
collection.delete_many({})

In [58]:
collection.insert_many(play_dict)

In [10]:
collection = db['players']

In [25]:
collection = db['players']

In [ ]:
list(collection.find({"_id":28003}))

# Load Scored-together

In [63]:
df = pd.read_csv('E:\magistrale\Data Managment\hw3\scored_together.csv')

In [66]:
df.drop('player_club_id.1', inplace=True, axis=1)

In [67]:
dict_t = df.to_dict('records')

In [71]:
dict_t[0]

{'player1': 72165,
 'player2': 67064,
 'player_club_id': 28095,
 'scored_together': 1,
 'game_id': 2506733,
 'pretty_name': 'Fk Ufa',
 '_id': ObjectId('628a92ac01764bc7d94dd8f5')}

In [70]:
db['scored_together'].insert_many(dict_t)

In [69]:
db['scored_together'].delete_many({})

# Query 1 - Find all the players that have played with at least 2 teams, and return the number of goals (at least 5) for each team. -

In [ ]:
with result as (
	select player_id, player_club_id, sum(goals) as goal_player
	from football_stats.appearances
	group by player_id, player_club_id
) 
select p.player_id, result.goal_player, club.pretty_name, p.pretty_name
from result result, football_stats_2.players p, 
	(select count(*) club_number, player_id 
	from result result, football_stats_2.clubs clubs
	where 1=1
    and clubs.club_id = result.player_club_id
    and goal_player > 5
	group by player_id 
	having club_number > 1) number_club_player,
football_stats_2.clubs club
where result.player_id = p.player_id
	and number_club_player.player_id = p.player_id
	and result.player_id = number_club_player.player_id
    and club.club_id = result.player_club_id
    and goal_player > 5
order by p.pretty_name  desc, goal_player desc
;

In [17]:
collection = db['players']
pip = [
        {"$match": {'all_club_id_player.1': {"$exists": True} }}, # see if the second element exists in the array
        {"$unwind":  '$appearances' },
        { "$group": 
                {
                    "_id":{"name_player":"$pretty_name", "club_name":"$appearances.club_name"},
                    "number_goal": { "$sum": "$appearances.goals" }
                }
            },
        {'$project': {"_id":0, 'name_player':'$_id.name_player', 'number_goal':"$number_goal", 'club_name':"$_id.club_name"}} ,
        {"$match": {'number_goal':{"$gt": 5}}},
        {"$sort":{'name_player': -1 }},  
        {"$limit":10}
]
pd.DataFrame(list(collection.aggregate(pip)))

,name_player,number_goal,club_name
0,Zlatan Ibrahimovic,66,Fc Paris Saint Germain
1,Zlatan Ibrahimovic,36,Ac Mailand
2,Zlatan Ibrahimovic,28,Manchester United
3,Zinho Gano,6,Kv Kortrijk
4,Zinho Gano,12,Sv Zulte Waregem
5,Zinho Gano,19,Waasland Beveren
6,Zinho Gano,6,Kv Oostende
7,Zian Flemming,17,Fortuna Sittard
8,Zian Flemming,6,Pec Zwolle
9,Zequinha,8,Vitoria Setubal Fc


# Query 2 - The top scorer (player with the highest number of goals) for each year

In [ ]:
with result as (
	select season, app.player_id, sum(goals) as goals
	from football_stats.games games, football_stats.appearances app
	where app.game_id = games.game_id
	group by season, app.player_id
)
select pretty_name, result.goals, result.season
from result result, football_stats.players player
where 1=1
	and result.player_id = player.player_id
	and (result.season, goals) in (select season, max(goals) from result group by season) 
	and result.season > 2013 
order by season
;

In [19]:
# at the momemnt in the insert we added season
collection = db['players']
pip = [
        {"$unwind":  '$appearances' },
        { "$group": 
                {
                   "_id":{"name_player":"$pretty_name", "season":"$appearances.season"},
                    "Number Goal": { "$sum": "$appearances.goals" }
                }
            },
    {"$project":{"_id":0, "player_name":'$_id.name_player', 'season':"$_id.season", "number_goal":"$Number Goal"}},
    { "$group": 
                {
                   "_id":{"season": "$season"},
                     "Number_goal": { "$max": "$number_goal" },
                    "gioc":{"$push":{"player_name": "$player_name", "number_goal": "$number_goal" }}, # show the meaning of this operation
                   

                }
            },
    {"$unwind":  '$gioc' },
    {"$project":{"_id":0, "season":"$_id.season", "matched":{"$eq":["$gioc.number_goal","$Number_goal" ]}, "number_goal":"$Number_goal", "player_name":"$gioc.player_name"}},
    {"$match":{"matched":True, "season":{"$gt":2013}}},
    {"$sort":{'season': 1 }},  
    {"$project":{"season":"$season", "number_goal":"$number_goal", "player_name":"$player_name"}},
    #{"$limit":3}
]
list(collection.aggregate(pip))

[{'season': 2014, 'number_goal': 61, 'player_name': 'Cristiano Ronaldo'},
 {'season': 2015, 'number_goal': 59, 'player_name': 'Luis Suarez'},
 {'season': 2016, 'number_goal': 54, 'player_name': 'Lionel Messi'},
 {'season': 2017, 'number_goal': 45, 'player_name': 'Lionel Messi'},
 {'season': 2018, 'number_goal': 51, 'player_name': 'Lionel Messi'},
 {'season': 2019, 'number_goal': 51, 'player_name': 'Robert Lewandowski'},
 {'season': 2020, 'number_goal': 52, 'player_name': 'Robert Lewandowski'},
 {'season': 2021, 'number_goal': 34, 'player_name': 'Robert Lewandowski'}]

## Query 3 - Average goal per game in the serie A and liga

In [ ]:
select comp.name, result.avg_goal
from (
	select competition_code,  sum(home_club_goals + away_club_goals) / count(*) as avg_goal
	from football_stats_2.games
    where competition_code in ("IT1", "ES1") 
    group by competition_code) as result,
    football_stats_2.competition comp
where comp.competition_id = result.competition_code

In [20]:
collection = db['competition']
pip = [ {"$match": { "_id" : { "$in": ["IT1","ES1"]}}},
        {"$unwind": "$games"},
        {"$lookup": {
            "from": "games", 
            "localField": "games",
            "foreignField": "_id",
            "as": "games"}},
       {"$unwind": "$games"},
       {"$group": { "_id":"$name", "Number_Goals_Home": { "$sum": "$games.home_club_goals" },"Number_Goals_Away": { "$sum": "$games.away_club_goals"},
                  "Number_Matches": { "$count": {}}}},
       {"$set": {"Total_goals" : {"$add": ["$Number_Goals_Home","$Number_Goals_Away"]}}},
       {"$set": {"Avg_goals": {"$divide": ["$Total_goals","$Number_Matches"]}}},
       {"$project": {"_id":0, "competition name":"$_id", "Avg_goals": "$Avg_goals"}},
     
      ]
pd.DataFrame(list(collection.aggregate(pip)))

,competition name,Avg_goals
0,serie-a,2.828195
1,laliga,2.647202


## Query 4 - The most aggressive country, we mean the nationality of the player -

In [ ]:
select country_of_birth, sum(case when red_cards > 0 then 2 else yellow_cards end) /count(*) as cards_per_game
from football_stats.appearances  cards, 
	football_stats.players 
where cards.player_id = players.player_id
group by country_of_birth
having count(*) > 200  # used to delete the result less significant
order by cards_per_game desc
limit 1;


In [21]:
collection = db['players']
pip = [ 
        {"$unwind":  '$appearances' },
        { "$group": 
                {
                   "_id":{"country_name":"$country_of_birth"}, # to have the number of the cards --> case when in sql
                    "cards" : { "$sum" : { "$cond": [ {  "$gt": [ "$appearances.red_cards" , 0] }, 2, "$appearances.yellow_cards" ] } },
                    "Number_match": {"$sum": 1}
                }
            },
       {"$match": {"Number_match": {'$gt': 200 } }},
       {"$set": {"cards_per_game": {"$divide": ["$cards","$Number_match"]}}},
       {"$sort":{'cards_per_game':-1}},
       {"$limit":1},
       {"$project":{"_id":0, "Country":"$_id.country_name", "cards":"$cards","cards_per_game":"$cards_per_game", "Number_match":"$Number_match"}}
      ]
pd.DataFrame(list(collection.aggregate(pip)))

,Country,cards,cards_per_game,Number_match
0,Bolivia,81,0.361607,224


## Query 5 - The most winning team for each championship with number of win

In [ ]:
with RESULT_QUERY as (
        select c.name as club_name, comp.name as competition_name, count(*) as Number_Win
        from ( # define the winner for each match
				SELECT (
                    case when home_club_goals > away_club_goals 
                        then home_club_id
                    else  away_club_id end
                    ) as winner, competition_code
             FROM football_stats.games 	 
             WHERE # to ignore drawn games
				   home_club_goals > away_club_goals    
                   or home_club_goals < away_club_goals
			) w, 
			football_stats.clubs c, 
			football_stats.competition comp
    where w.winner = c.club_id
    and comp.competition_id = w.competition_code
    group by c.name, comp.name
    having count(*) > 5        
    )
	
	# and now we should extract for each championship the most winning team
    # if we try to find it before, there is an error because we grouped by competition name and number win and so we can't 
    select * from RESULT_QUERY q1 
		# in this case we admit the duplicates in case of two teams having the same number of wins
		where (q1.competition_name,q1.Number_win) = 
		(
			# we extract the max number of win for each competition with the competition name
			
			SELECT competition_name, MAX(Number_win)
			FROM RESULT_QUERY q2
			where q1.competition_name = q2.competition_name
			group by competition_name
		)
	order by number_win desc
    ;

In [22]:
collection = db['competition']
pip = [ 
        {"$unwind":  '$games' },
        {"$lookup": {"from": "games", 
                    "localField": "games", 
                    "foreignField": "_id", 
                    "as": "games"}},
        {"$unwind": "$games"},
        {"$match": 
        {"$expr": {"$ne": ["$games.home_club_goals", "$games.away_club_goals"] } }}, # to discard the draws
      
        # to have the winner of the match --> case when in sql
      {"$project": { "winner": {"$cond": [ {  "$gt": [ "$games.home_club_goals" , "$games.away_club_goals"] }, "$games.home_club_name", "$games.away_club_name" ] } , "comp_name":"$name"}},
       { "$group": 
                {
                   "_id": {"winner":"$winner", "comp_name": "$comp_name"},
                    "Number_match": {"$sum": 1},
                    
                }
            },
       { "$group": 
                {
                   "_id": {"comp_name": "$_id.comp_name"},
                    "max_win": {"$max": "$Number_match"},
                    "win":{"$push":{"winner": "$_id.winner", "number_win": "$Number_match" }},
                }
            },
      {"$unwind":  '$win' },
      {"$match": {"$expr": {'$eq': [ "$max_win", "$win.number_win"], }}},
      {"$project": {"_id":0, "best club competition": "$win.winner", "competition": "$_id.comp_name", "win":'$max_win'}},
      {"$sort":{'win': -1}},
      {"$limit":10}
      ]
pd.DataFrame(list(collection.aggregate(pip)))

,best club competition,competition,win
0,Juventus Turin,serie-a,203
1,Fc Paris Saint Germain,ligue-1,203
2,Manchester City,premier-league,201
3,Celtic Glasgow,scottish-premiership,199
4,Fc Barcelona,laliga,199
5,Benfica Lissabon,liga-portugal-bwin,195
6,Fc Bayern Munchen,bundesliga,195
7,Ajax Amsterdam,eredivisie,185
8,Olympiakos Piraus,super-league-1,171
9,Besiktas Istanbul,super-lig,164


## Query 6 - The italian players that had at least 100 matches in foreign championships - 

In [ ]:
select players.pretty_name, count(*) as number_matches_abroad
from  football_stats_2.appearances app,football_stats_2.players,  football_stats_2.competition comp
where 1=1
and players.country_of_birth = "Italy"
and app.player_id = players.player_id 
and comp.competition_id = app.competition_id
and comp.country_name != "Italy" and comp.type = "first_tier"
group by players.pretty_name
having count(*) > 100
order by number_matches_abroad desc;

In [9]:
db['players'].create_index([('country_of_birth', -1)], name='country_index')

'country_index'

In [23]:
collection = db['players']
pip = [{"$match": {"country_of_birth": "Italy"}},
       {"$unwind" : "$appearances"},
       {"$group": { "_id":{"Player_name": "$pretty_name","Competition_id": "$appearances.competition_id"}, "Number_matches": { "$count": {} }}},
       {"$project": {"_id":0,"pretty_name":"$_id.Player_name", "comp_id": "$_id.Competition_id", "Number_matches":"$Number_matches"}},
        {"$lookup": {
            "from": "competition", 
            "let":{"comp_id": "$comp_id"},
            "pipeline":[
                 {"$match": {
                        "$expr": {
                                 "$and":[
                                 { "$eq": ["$_id", "$$comp_id"]} , 
                                 {"$ne": ["$country_name", "Italy"]}, 
                                 {"$eq": ["$type","first_tier"]}
                                 ]
                        }
                 }
                },
                { "$project": { "country_name": "$country_name", "type":"$type"}}
            ],
            "as": "competition"}},
       {"$unwind": "$competition"},
       {"$group": { "_id":{"Player_name": "$pretty_name"}, "Number_matches": { "$sum": "$Number_matches" }}},
       
       {"$sort":{"Number_matches":-1}},
       {"$limit":3},
       {"$project":{"_id":0, "Player_name":"$_id.Player_name", "Number_matches":"$Number_matches"}}
      ]
pd.DataFrame(list(collection.aggregate(pip)))

,Player_name,Number_matches
0,Marco Verratti,179
1,Angelo Ogbonna,173
2,Thiago,169


## Query 7 - Ranking of pairs of player that scored together in the same match - 

In [ ]:
select ap1.player_id as player1, ap2.player_id as player2, ap1.game_id, case when ap1.goals > 0 and ap2.goals > 0 then True else False end as scored_together
from football_stats.appearances ap1, football_stats.appearances ap2
where 1=1
    and  ap1.game_id = ap2.game_id
    and ap1.player_club_id = ap2.player_club_id
    and ap1.player_id > ap2.player_id    #to remove the duplicates generated by the self-join es. (player1-player2 and also player2-player1)
    # and ap1.player_id != ap2.player_id
    and ap1.goals > 0 and ap2.goals > 0

In [24]:
collection = db['scored_together']
pip = [
       {"$group": { "_id":{"player1":"$player1", "player2":"$player2", "club_name":"$pretty_name"}, "scored_together": { "$sum": "$scored_together" }}},
       {"$lookup": {
            "from": "players", 
            "let":{"player1": "$_id.player1"},
            "pipeline":[
                 {"$match": {
                        "$expr": 
                            
                                 { "$eq":["$_id", "$$player1"]} ,             
                    }
                },
                { "$project": { "pretty_name": "$pretty_name"}}
            ],
            "as": "player1"}},
     {"$lookup": {
            "from": "players", 
            "let":{"player2": "$_id.player2"},
            "pipeline":[
                 {"$match": {
                        "$expr": 
                            
                                 { "$eq":["$_id", "$$player2"]} ,             
                    }
                },
                { "$project": { "pretty_name": "$pretty_name"}}
            ],
            "as": "player2"}},
    
        {"$sort":{"scored_together":-1}},
       {"$limit": 10},
     {"$unwind":"$player1"},
     {"$unwind":"$player2"},
     { "$project": {"_id":0, "player1": "$player1.pretty_name", "player2":"$player2.pretty_name", "scored_together":"$scored_together", "club_name":"$_id.club_name"}},
    {"$limit":10}
      ]
pd.DataFrame(list(collection.aggregate(pip)))

,player1,player2,scored_together,club_name
0,Luis Suarez,Lionel Messi,81,Fc Barcelona
1,Thomas Muller,Robert Lewandowski,61,Fc Bayern Munchen
2,Neymar,Lionel Messi,43,Fc Barcelona
3,Sadio Mane,Mohamed Salah,42,Fc Liverpool
4,Neymar,Luis Suarez,35,Fc Barcelona
5,Karim Benzema,Cristiano Ronaldo,34,Real Madrid
6,Serge Gnabry,Robert Lewandowski,32,Fc Bayern Munchen
7,Harry Kane,Heung Min Son,32,Tottenham Hotspur
8,Mohamed Salah,Roberto Firmino,29,Fc Liverpool
9,Kingsley Coman,Robert Lewandowski,29,Fc Bayern Munchen


## Query 8 - Ranking of the players with the higher number of goals and less aggressive in term of the number of cards 

In [ ]:
select app.player_id, app.number_goals, player.pretty_name
from football_stats_2.players player, (select player_id, sum(goals) as number_goals, sum(yellow_cards) as number_yellow, sum(red_cards) as number_red
from football_stats_2.appearances
group by player_id) app
where 1=1
and player.player_id  = app.player_id 
and (number_yellow <11 or number_red <2 )
order by app.number_goals desc

In [25]:
collection = db['players']
pip = [
       {"$unwind" : "$appearances"},
       {"$group": { "_id":"$pretty_name", "Number_goals": { "$sum": "$appearances.goals" }, "Num_yellow_cards": { "$sum": "$appearances.yellow_cards" },
                  "Num_red_cards": { "$sum": "$appearances.red_cards" }}},
       {"$match": {"$or":[{"Num_yellow_cards":{"$lt":11}},{"Num_red_cards":{"$lt":2}}]}}, 
       {"$project": {"Name":"$pretty_name","Number goals":"$Number_goals"}},
       {"$sort" : {"Number goals": -1 }},
       {"$limit": 20},
       {"$project":{"_id":0, "Player_name":"$_id", "Number goals":"$Number goals"}}
      ]
pd.DataFrame(list(collection.aggregate(pip)))

,Player_name,Number goals
0,Robert Lewandowski,328
1,Lionel Messi,324
2,Luis Suarez,238
3,Harry Kane,213
4,Pierre Emerick Aubameyang,211
5,Karim Benzema,192
6,Mohamed Salah,189
7,Sergio Aguero,174
8,Romelu Lukaku,173
9,Edinson Cavani,168


# Query 9 - The three shortest players (less than 170 cm) with higher number of goals - 

In [ ]:
select pretty_name, number_goal
from (select player_id, sum(goals) as number_goal
    from football_stats_2.appearances
    group by player_id) result,
football_stats_2.players player
where result.player_id = player.player_id 
and height_in_cm < 170
order by number_goal desc
limit 3;

In [26]:
collection = db['players']
pip = [
        {"$match": {   "height_in_cm":{"$lt":170} }},
        {"$unwind":  '$appearances' },
        { "$group": 
                {
                    "_id":"$pretty_name",
                    "Number Goal": { "$sum": "$appearances.goals" }
                }
            },
        {"$sort":{'Number Goal': -1, }},    

        {"$limit": 3}
]
pd.DataFrame(list(collection.aggregate(pip)))

,_id,Number Goal
0,Lionel Messi,324
1,Dries Mertens,131
2,Lorenzo Insigne,102


# Query 10 - Top 10 players in Serie A with higher number of goals away against teams that were higher in rank

In [ ]:
select p.pretty_name, sum(app.goals) as number_goals
from football_stats_2.players p, 
	 football_stats_2.appearances app, 
     football_stats_2.games g
where p.player_id = app.player_id
and app.game_id = g.game_id
and app.player_club_id = g.away_club_id
and g.away_club_position > g.home_club_position
and app.competition_id = "IT1" #index here
and app.goals > 0
group by p.pretty_name
order by number_goals desc
limit 10;

In [27]:
collection = db['players']

pip = [
        {"$unwind":  '$appearances' }, 
        {"$match":{
                    "$and":
                   [{"appearances.goals":{"$gt":0}},
                    {"appearances.competition_id":"IT1"},
                   ]
                   }
        },
        {"$project": {"_id":0,"games_id":"$appearances.game_id", "player_club_id": "$appearances.player_club_id", "pretty_name":"$pretty_name", "number_goal":"$appearances.goals"}},
        {"$lookup": {
            "from": "games", 
            "let": {"game_id":"$games_id","player_club":"$player_club_id"},
            "pipeline":[
                {"$match": {
                        "$expr": { 
                            "$and":[
                                 {"$eq": ["$_id", "$$game_id"]} ,
                                 {"$eq": ["$away_club_id", "$$player_club"]},
                                 {"$gt": ["$away_club_position","$home_club_position" ]}
                            ]
                        }
                    }
                },
            ],
            
            "as": "games2"
            
        }},
      
        {"$match": {"games2.0": {"$exists": True}}},
        {"$group": { "_id": "$pretty_name", "Number_goals": { "$sum": "$number_goal" }}},
        {"$sort" : {"Number_goals": -1 }},
        {"$limit": 10},
        {"$project":{"_id":0, "Player_name":"$_id", "Number_goals":"$Number_goals"}}
]
pd.DataFrame(list(collection.aggregate(pip)))

,Player_name,Number_goals
0,Andrea Belotti,24
1,Fabio Quagliarella,18
2,Gregoire Defrel,15
3,Domenico Berardi,15
4,Simy,15
5,Joao Pedro,14
6,Andrea Petagna,13
7,Mattia Destro,11
8,Ciro Immobile,11
9,Leonardo Pavoletti,11
